## Monitoring SQL Server with Python (Remotely)

In [ ]:
import pyodbc
from prettytable import PrettyTable

def connect (Server_name, database):
    conn = pyodbc.connect (
        'DRIVER={SQL Server};'
        'SERVER=' + Server_name + ';'
                                  'DATABASE=' + database + ';'
                                                           'Trusted_Connection=yes;')
    cursor = conn.cursor ()
    return cursor


if __name__ == "__main__":
    while True:
        val = int (input (
            "1. Execute query manually \n2. Blocking session \n3. Long running sessions \n4. Curent running sessions \n5. WhoisActive \n6. Open Transactions \n7. Check Log Space \n8. Databases Details \n9. DatabaseSize in GB \n10. Server Details \n11. Exit\n\nEnter your option: "))
        if val == 1:
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            query = '''input ("Enter query: ")'''
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()
            print ("output_query: ", output_query)

        elif val == 2:
            query = '''sp_who2;'''
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()
            if output_query:

                my_table = PrettyTable ()
                my_table.field_names = ["SPID", "Status", "Login", "Hostname", "Blkby", "Database_Name", "Command",
                                        "CPU_Time", "Disk_IO", "LastBatch", "ProgramName", "Spid", "RequestID"]

                for item in output_query:
                    my_table.add_row (item)

                # for item in output_query[0]:

                print (my_table)

            else:

                print ("No data returned")

        elif val == 3:
            query = '''SELECT TOP 20
                              qs.total_elapsed_time / qs.execution_count / 1000000.0 AS average_seconds,
                              qs.total_elapsed_time / 1000000.0 AS total_seconds,
                              qs.execution_count,
        					  o.name AS object_name,
                              DB_NAME(qt.dbid) AS database_name,
                              SUBSTRING (qt.text,qs.statement_start_offset/2, 
                              (CASE WHEN qs.statement_end_offset = -1 
                              THEN LEN(CONVERT(NVARCHAR(MAX), qt.text)) * 2 
                              ELSE qs.statement_end_offset END - qs.statement_start_offset)/2) AS individual_query
        					  FROM sys.dm_exec_query_stats qs
                              CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) as qt
                              LEFT OUTER JOIN sys.objects o ON qt.objectid = o.object_id
                              where qt.dbid = DB_ID()
                              ORDER BY average_seconds DESC;'''
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()

            if output_query:
                my_table = PrettyTable ()
                my_table.field_names = ["Average_Seconds.", "Total_Seconds", "Execution_Count", "Query", "Object_Name",
                                        "Database_Name"]
                for item in output_query:
                    my_table.add_row (item)

                # for item in output_query[0]:
                print (my_table)
            else:
                print ("No data returned")


        elif val == 4:
            query = '''SELECT r.start_time [Start Time],
        			session_ID [SPID],
                    DB_NAME(database_id) [Database],
                    Status,command,wait_type,wait_time,wait_resource,
                    last_wait_type,
        			SUBSTRING(t.text,(r.statement_start_offset/2)+1,
                    CASE WHEN statement_end_offset=-1 OR statement_end_offset=0
                    THEN (DATALENGTH(t.Text)-r.statement_start_offset/2)+1
                    ELSE (r.statement_end_offset-r.statement_start_offset)/2+1
                    END) [Executing SQL]
        			FROM        sys.dm_exec_requests r
        			OUTER APPLY sys.dm_exec_sql_text(sql_handle) t
        			WHERE       session_id != @@SPID -- don't show this query
        			AND         session_id > 50 -- don't show system queries
        			ORDER BY    r.start_time'''
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()

            if output_query:
                my_table = PrettyTable ()
                my_table.field_names = ["StartTime", "Spid", "Database", "Status", "Command", "Wait_Type", "Wait_Time",
                                        "Wait_resource", "Last_wait_type", "Execute_SQL"]
                for item in output_query:
                    my_table.add_row (item)

                # for item in output_query[0]:
                print (my_table)
            else:
                print ("No data returned")

        elif val == 5:
            query = '''sp_whoisactive'''
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()

            if output_query:
                my_table = PrettyTable ()
                my_table.field_names = ["Date", "Session_ID", "SQL_TEXT", "Login_Name", "Wait_INFO", "CPU",
                                        "Temp_DB_Allocations",
                                        "Temp_DB_Current", "Blocking SID", "Reads", "Writes", "Physical_reads",
                                        "Used_Memory", "Status", "Open_Tran_Count",
                                        "Percent_Complete", "Hostname", "DB_Name", "Program_name", "Start_Time",
                                        "Login_Time",
                                        "Request_ID ", "Collection_Time"]
                for item in output_query:
                    my_table.add_row (item)

                # for item in output_query[0]:
                print (my_table)
            else:
                print ("No data returned")

        elif val == 6:
            query = '''SELECT spid,
       PROGRAM_NAME,
       nt_userName,
       loginame,
       DB_NAME(s.dbid) AS DatabaseName   
	   FROM   sysprocesses s
       WHERE  open_tran = 1'''
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()

            if output_query:
                my_table = PrettyTable ()
                my_table.field_names = ["Spid", "Program_name","Username","Login_name","Database_Name"]
                for item in output_query:
                    my_table.add_row (item)

                # for item in output_query[0]:
                print (my_table)
            else:
                print ("No data returned")

        elif val == 7:
            query = '''DBCC SQLPERF(logspace);'''
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()

            if output_query:
                my_table = PrettyTable ()
                my_table.field_names = ["Database_Name", "LogSize_MB", "Logsize_Used_Percentage", "Status"]
                for item in output_query:
                    my_table.add_row (item)

                # for item in output_query[0]:
                print (my_table)
            else:
                print ("No data returned")


        elif val == 8:
            query = '''sp_helpdb'''
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()

            if output_query:
                my_table = PrettyTable ()
                my_table.field_names = ["Name.", "DB_Size", "Owner", "DB_ID", "Created", "Status",
                                        "Compatibility_Level"]
                for item in output_query:
                    my_table.add_row (item)

                # for item in output_query[0]:
                print (my_table)
            else:
                print ("No data returned")

        elif val == 9:
            query = '''SELECT d.NAME
        				,ROUND(SUM(CAST(mf.size AS bigint)) * 8 / 1024, 0) Size_MBs
        				,(SUM(CAST(mf.size AS bigint)) * 8 / 1024) / 1024 /1 AS Size_GBs
        				FROM sys.master_files mf
        				INNER JOIN sys.databases d ON d.database_id = mf.database_id
        				WHERE d.database_id > 4 -- Skip system databases
        				GROUP BY d.NAME
        				ORDER BY d.NAME
        				;'''
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()

            if output_query:
                my_table = PrettyTable ()
                my_table.field_names = ["Name", "Size_in_MB", "Size_in_GB"]
                for item in output_query:
                    my_table.add_row (item)

                # for item in output_query[0]:
                print (my_table)
            else:
                print ("No data returned")

        elif val == 10:
            query = '''exec master..xp_cmdshell 'systeminfo';'''
            server = input ("Enter Server Name: ")
            database = input ("Enter Database Name: ")
            cur = connect (server, database)
            cur.execute (query)
            output_query = cur.fetchall ()

            if output_query:
                my_table = PrettyTable ()
                my_table.field_names = ["Output"]
                for item in output_query:
                    my_table.add_row (item)

                # for item in output_query[0]:
                print (my_table)
            else:
                print ("No data returned")


        elif val == 11:
            exit()
        else:
            print("Invalid Input")

1. Execute query manually 
2. Blocking session 
3. Long running sessions 
4. Curent running sessions 
5. WhoisActive 
6. Open Transactions 
7. Check Log Space 
8. Databases Details 
9. DatabaseSize in GB 
10. Server Details 
11. Exit

Enter your option: 2
Enter Server Name: localhost
Enter Database Name: master
+-------+--------------------------------+---------------------------+-------------+-------+---------------+----------------------------+----------+---------+----------------+----------------------------------+-------+-----------+
|  SPID |             Status             |           Login           |   Hostname  | Blkby | Database_Name |          Command           | CPU_Time | Disk_IO |   LastBatch    |           ProgramName            |  Spid | RequestID |
+-------+--------------------------------+---------------------------+-------------+-------+---------------+----------------------------+----------+---------+----------------+----------------------------------+-------+-------